In [ ]:
from langchain_openai import ChatOpenAI
from typing import Annotated,TypedDict
from langgraph.graph.message import add_messages
from langchain_teddynote.tools.tavily import TavilySearch
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import StateGraph,START,END
from langgraph.checkpoint.memory import MemorySaver

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.tools.retriever import create_retriever_tool


In [ ]:

class State(TypedDict):
    messages:Annotated[list,add_messages]


In [ ]:
# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("../data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성합니다.
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성합니다.
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

In [ ]:

search_tool = TavilySearch()

tools = [search_tool,retriever_tool]

tool_node = ToolNode(tools)

llm = ChatOpenAI(model='gpt-4.1-mini',temperature=0)

llm_with_bind = llm.bind_tools(tools)

def chatbot(state:State):
    action = llm_with_bind.invoke(state['messages'])
    print(action)
    return State({'messages':[action]})

state_graph = StateGraph(State)

state_graph.add_node('chatbot',chatbot)
state_graph.add_node('tools',tool_node)

state_graph.add_edge(START,'chatbot')
state_graph.add_conditional_edges(
    source='chatbot',
    path=tools_condition
)
state_graph.add_edge('tools','chatbot')
state_graph.add_edge('chatbot',END)
graph = state_graph.compile()


In [ ]:
# 메모리 저장소 생성
graph = state_graph.compile(checkpointer=MemorySaver())

In [ ]:
from langchain_teddynote.graphs import visualize_graph

# 그래프 시각화
visualize_graph(graph)

In [ ]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    recursion_limit=10,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
    configurable={"thread_id": "1"},  # 스레드 ID 설정
)

In [ ]:
query = "2025년 하반기 행사"
for event in graph.stream({"messages": [("user", query)]}, config=config):
    for value in event.values():
        value["messages"][-1].pretty_print()

In [ ]:
query = "내부 문서중 삼성 AI 모델 정보"
for event in graph.stream({"messages": [("user", query)]}, config=config):
    for value in event.values():
        value["messages"][-1].pretty_print()

In [ ]:
query = "안녕/"
for event in graph.stream({"messages": [("user", query)]}, config=config):
    for value in event.values():
        value["messages"][-1].pretty_print()

In [ ]:
snapshot = graph.get_state(config)
snapshot.values